In [ ]:
%pip install mediapipe
%pip install tensorflow
# Add other necessary packages

In [ ]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np
import csv

from mediapipe.tasks import python
from mediapipe.tasks.python import audio

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Function to perform facial landmarking on an image
def perform_facial_landmarking(image_path):
    # Read image
    image = cv2.imread(image_path)

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image
    results = face_mesh.process(rgb_image)

    # Extract landmarks
    landmarks = []
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                landmarks.append([x, y, z])

    return np.array(landmarks)

In [ ]:
# # Folder containing FER2013 dataset images
# dataset_folder = 'dataset/train'  # Adjust the folder path accordingly
# csv_file = 'fer2013_landmarks.csv'

# # Apply facial landmarking to all images in subfolders
# facial_landmarks_data = []

# # # Check if the CSV file already exists
# # if not os.path.isfile(csv_file):
# #     # Create a new CSV file with header if it doesn't exist
# #     with open(csv_file, 'w', newline='') as file:
# #         writer = csv.writer(file)
# #         # column names are emotion, image path
# #         header = ['emotion', 'image_path'] + [f'x_{i}' for i in range(468)]  # Assuming 468 landmarks
# #         print(header)
# #         writer.writerow(header)

# # Append facial landmarks data to the CSV file
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Write header
#     header = ['emotion', 'image_path'] + [f'x_{i}' for i in range(468)]  # Assuming 468 landmarks
#     print(header)
#     writer.writerow(header)

#     # Write data
#     for emotion_folder in os.listdir(dataset_folder):
#         emotion_path = os.path.join(dataset_folder, emotion_folder)
#         print('Processing emotion:', emotion_folder)
#         if os.path.isdir(emotion_path):
#             for image_file in os.listdir(emotion_path):
#                 image_path = os.path.join(emotion_path, image_file)
#                 landmarks = perform_facial_landmarking(image_path)
#                 print('landmarks: ', landmarks)

#                 # Flatten the list of landmarks and group by 3 (x, y, z)
#                 flattened_landmarks = [item for sublist in landmarks for item in sublist]
#                 # grouped_landmarks = [flattened_landmarks[i:i+3] for i in range(0, len(flattened_landmarks), 3)]

#                 # Append data to CSV
#                 row = [emotion_folder, image_path] + flattened_landmarks
#                 writer.writerow(row)
#                 print('CSV Row:', row)  # Print the CSV row


#                 # Open CSV in read mode to print its content
#                 # with open(csv_file, 'r', newline='') as read_file:
#                 #     csv_reader = csv.reader(read_file)
#                 #     csv_content = list(csv_reader)
#                 #     print('Current CSV Content:', csv_content)


# # for emotion_folder in os.listdir(dataset_folder):
# #     emotion_path = os.path.join(dataset_folder, emotion_folder)
# #     print('Processing emotion:', emotion_folder)
# #     if os.path.isdir(emotion_path):
# #         for image_file in os.listdir(emotion_path):
# #             image_path = os.path.join(emotion_path, image_file)
# #             landmarks = perform_facial_landmarking(image_path)
# #             print(landmarks)  # x y z
#             # facial_landmarks_data.append({'landmarks': landmarks.tolist(), 'emotion': emotion_folder, 'image_path': image_path})

In [ ]:
NUM_LANDMARKS = 468  # According to MediaPipe FaceMesh documentation

# Folder containing FER2013 dataset images
dataset_folder = 'dataset/train'  # Adjust the folder path accordingly
csv_file = 'fer2013_landmarks.csv'

# Apply facial landmarking to all images in subfolders
facial_landmarks_data = []

In [ ]:


# # Check if the CSV file already exists
# if not os.path.isfile(csv_file):
#     # Create a new CSV file with header if it doesn't exist
#     with open(csv_file, 'w', newline='') as file:
#         writer = csv.writer(file)
#         # column names are emotion, image path
#         header = ['emotion', 'image_path'] + [f'x_{i}' for i in range(468)]  # Assuming 468 landmarks
#         print(header)
#         writer.writerow(header)

# Append facial landmarks data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write header
    coordinate_columns = [f'{axis}{i+1}' for i in range(468) for axis in ['x', 'y', 'z']]
    header = ['emotion', 'image_path'] + coordinate_columns
    print(header)
    writer.writerow(header)

    # Write data
    for emotion_folder in os.listdir(dataset_folder):
        emotion_path = os.path.join(dataset_folder, emotion_folder)
        print('Processing emotion:', emotion_folder)
        if os.path.isdir(emotion_path):
            for image_file in os.listdir(emotion_path):
                image_path = os.path.join(emotion_path, image_file)
                landmarks = perform_facial_landmarking(image_path)
                print('landmarks: ', landmarks)

                # Flatten the list of landmarks and group by 3 (x, y, z)
                flattened_landmarks = [item for sublist in landmarks for item in sublist]
                # grouped_landmarks = [flattened_landmarks[i:i+3] for i in range(0, len(flattened_landmarks), 3)]

                # Append data to CSV
                row = [emotion_folder, image_path] + flattened_landmarks
                writer.writerow(row)
                print('CSV Row:', row)  # Print the CSV row


                # Open CSV in read mode to print its content
                # with open(csv_file, 'r', newline='') as read_file:
                #     csv_reader = csv.reader(read_file)
                #     csv_content = list(csv_reader)
                #     print('Current CSV Content:', csv_content)


# for emotion_folder in os.listdir(dataset_folder):
#     emotion_path = os.path.join(dataset_folder, emotion_folder)
#     print('Processing emotion:', emotion_folder)
#     if os.path.isdir(emotion_path):
#         for image_file in os.listdir(emotion_path):
#             image_path = os.path.join(emotion_path, image_file)
#             landmarks = perform_facial_landmarking(image_path)
#             print(landmarks)  # x y z
            # facial_landmarks_data.append({'landmarks': landmarks.tolist(), 'emotion': emotion_folder, 'image_path': image_path})

In [ ]:
# ## UPDATE LANDMARKS

# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)

#     # Write header
#     header = ['emotion', 'image_path'] + [f'x{i+1}' for i in range(468)] + [f'y{i+1}' for i in range(468)] + [f'z{i+1}' for i in range(468)]  # Assuming 468 landmarks


In [ ]:
print(facial_landmarks_data)

# Convert the result into a DataFrame
landmarks_df = pd.DataFrame(facial_landmarks_data)

# Save the DataFrame with facial landmarks, emotions, and image paths to a CSV file
landmarks_df.to_csv(csv_file, index=False)

# Training the Model

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('landmarked_dataset.csv')

In [ ]:
# Pre-process the data
X = df.drop('emotion_label', axis=1).values
y = df['emotion_label'].values

In [ ]:
# Encode Labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Make predictions
predictions = model.predict(X_test)
